<a href="https://colab.research.google.com/github/qn19325/individualProject/blob/main/annFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from tqdm.notebook import tqdm
from google.colab import drive

In [ ]:
drive.mount('/content/gdrive')
!unzip '/content/gdrive/MyDrive/verticalEnv.zip' -d '/content/trainingData'

Mounted at /content/gdrive
Archive:  /content/gdrive/MyDrive/verticalEnv.zip
   creating: /content/trainingData/verticalEnv/
  inflating: /content/trainingData/verticalEnv/.DS_Store  
  inflating: /content/trainingData/__MACOSX/verticalEnv/._.DS_Store  
  inflating: /content/trainingData/verticalEnv/64x64.csv  
  inflating: /content/trainingData/__MACOSX/verticalEnv/._64x64.csv  
   creating: /content/trainingData/verticalEnv/64x64/
  inflating: /content/trainingData/verticalEnv/animation.py  
  inflating: /content/trainingData/__MACOSX/verticalEnv/._animation.py  
  inflating: /content/trainingData/verticalEnv/movie.mp4  
  inflating: /content/trainingData/__MACOSX/verticalEnv/._movie.mp4  
  inflating: /content/trainingData/verticalEnv/64x64/348.png  
  inflating: /content/trainingData/verticalEnv/64x64/412.png  
  inflating: /content/trainingData/verticalEnv/64x64/374.png  
  inflating: /content/trainingData/verticalEnv/64x64/360.png  
  inflating: /content/trainingData/verticalEnv/

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
!pip install wandb --upgrade
import wandb 
wandb.login()

     |████████████████████████████████| 1.8 MB 4.2 MB/s 
     |████████████████████████████████| 181 kB 35.2 MB/s 
     |████████████████████████████████| 144 kB 46.9 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=346c2506b9859f1442074bb01178dabc5968c6be163a43dbe4ba167a9fc56e2d
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [ ]:
config = dict(
    epochs=1000,
    train_batch_size=8,
    test_batch_size=1,
    learning_rate=0.001,
    input_size = 4096,
    hidden_size = 128,
    output_size = 1,
    dataset="vertical_64x64",
    architecture="ANN")

In [ ]:
def make(config):
    # Make the RNN model
    model = ANN(config).to(device)

    # Make the loss and optimizer
    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), 
                                 lr=config.get("learning_rate"))
    
    return model, criterion, optimizer

In [ ]:
class ImageDataLoader(Dataset):
  def __init__(self, dir_=None):
    self.data_df = pd.read_csv('trainingData/verticalEnv/64x64.csv')
    self.dataset_len = len(self.data_df) # read the number of len of your csv files
  def __getitem__(self, idx):
    # load the next image
    f_name_t = self.data_df['Filename'][idx]
    f_name_tp1 = self.data_df['Filename'][idx+1]
    label = self.data_df['Label'][idx]
    label = label.astype(np.float32) 
    label = np.true_divide(label, 20)
    img_t = torchvision.io.read_image('trainingData/verticalEnv/64x64/{}'.format(f_name_t))
    img_tp1 = torchvision.io.read_image('trainingData/verticalEnv/64x64/{}'.format(f_name_tp1))
    img_t = img_t.float().div_(255.0)
    img_tp1 = img_tp1.float().div_(255.0)
    # invert pixel values
    img_t = 1 - img_t
    img_tp1 = 1 - img_tp1
    # crop
    img_t = torchvision.transforms.functional.crop(img_t, 0, 0, 32, 64)
    img_tp1 = torchvision.transforms.functional.crop(img_tp1, 0, 0, 32, 64)
    img_t = img_t.flatten()
    img_tp1 = img_tp1.flatten()
    input_seq = torch.cat((img_t, img_tp1), dim=0)
    return input_seq, label
  def __len__(self):
    return self.dataset_len - 1

In [ ]:
dataset = ImageDataLoader()
train_loader = DataLoader(dataset=dataset, batch_size=config.get("batch_size"), shuffle=True)

In [ ]:
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
train_dataset_loader = DataLoader(dataset=train_dataset, batch_size=8, shuffle=True)
test_dataset_loader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=True)

In [ ]:
class ANN(nn.Module):
    def __init__(self, config):
        super(ANN, self).__init__()
        self.fc1 = nn.Linear(config.get("input_size"), config.get("hidden_size"))   # hidden layer
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(config.get("hidden_size"), config.get("output_size"))   # output layer

    def forward(self, x):
        x = nn.functional.relu(self.fc1(x))      # activation function
        x = self.relu(x)
        x = self.fc2(x)             # linear output
        return x

In [ ]:
def train(inputSeq, labels, model, optimizer, criterion, epoch):
  model.train()

  inputSeq, labels = inputSeq.to(device), (labels.float()).to(device)
  
  # # Forward pass ➡
  print(inputSeq.shape)

  outputs = model(inputSeq.to(device))
  loss = criterion(outputs, labels.float())

  # Backward pass ⬅
  optimizer.zero_grad()
  loss.backward()

  # Step with optimizer
  optimizer.step()

  return loss

In [ ]:
def test(model, epoch):
  model.eval()

  # Run the model on some test examples
  with torch.no_grad():
    correct, total = 0, 0
    for inputSeq, labels in test_dataset_loader:
      inputSeq, labels = inputSeq.to(device), (labels.float()).to(device)

      outputs = model(inputSeq.to(device))
        
      predicted = round(outputs.item(), 1)
      speed = round(labels.item(), 1)
      
      total += labels.size(0)
      if predicted == speed:
        correct += 1


      # predicted = torch.round(outputs[:,-1].squeeze(), decimals=1)
      # speed = torch.round(labels, decimals=1)
      
      # equal = torch.eq(predicted, speed)
      
      # num_correct = torch.count_nonzero(equal)
      
      # total += labels.size(0)
      # correct += num_correct.item()

    print(f"Accuracy of the model at epoch {epoch}: " + f"{100 * correct / total}%")
    
    wandb.log({"test_accuracy": correct / total})

In [ ]:
def train_log(loss, batch_ct, example_ct, epoch):
  # Where the magic happens
  wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
  print(f"Epoch: " + str(epoch) + f", Batch: " + str(batch_ct).zfill(4) + f", Example: " + str(example_ct).zfill(5) + f", Loss: {loss:.3f}")

In [ ]:
def main(model, criterion, optimizer, config):
  # Tell wandb to watch what the model gets up to: gradients, weights, and more!
  wandb.watch(model, criterion, log="all", log_freq=10)

  # Run training and track with wandb
  example_ct = 0
  batch_ct = 0
  # mm_image = None
  for epoch in tqdm(range(config.get("epochs"))):
    for idx, (inputSeq, labels) in enumerate(train_loader):
      
      loss = train(inputSeq, labels, model, optimizer, criterion, epoch)

      example_ct += len(inputSeq)
      batch_ct += 1

      # Report metrics every 25th batch
      if ((batch_ct + 1) % 25) == 0:
        train_log(loss, batch_ct, example_ct, epoch)

    if epoch % 5 == 0:
        test(model, epoch)

    # if epoch % 25 == 0:
    #   EPOCH = epoch
    #   PATH = "gdrive/MyDrive/models/ann_" + str(epoch) + ".pt"
    #   LOSS = loss
    #   torch.save({
    #               'epoch': EPOCH,
    #               'model_state_dict': model.state_dict(),
    #               'optimizer_state_dict': optimizer.state_dict(),
    #               'loss': LOSS,
    #               }, PATH)

In [ ]:
def training_testing(hyperparameters):
  # tell wandb to get started
  with wandb.init(project="individual_project", config=hyperparameters):
    # access all HPs through wandb.config, so logging matches execution!
    config = wandb.config

    # make the model, data, and optimization problem
    model, criterion, optimizer = make(config)
    print(model)

    main(model, criterion, optimizer, config)

  return model

In [ ]:
model = training_testing(config)

ANN(
  (fc1): Linear(in_features=4096, out_features=128, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=128, out_features=1, bias=True)
)


  0%|          | 0/1000 [00:00<?, ?it/s]

torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
Epoch: 0, Batch: 0024, Example: 98304, Loss: 0.165
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:529: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
Epoch: 0, Batch: 0049, Example: 200704, Loss: 0.282
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
Epoch: 0, Batch: 0074, Example: 303104, Loss: 0.027
torch.Size([4096])
torch.Size([4096])
torch.Size([4096])
tor